In [1]:
%load_ext autoreload
%autoreload 2

import data
import config

In [2]:
%%bash

echo "--lines--"
head ./data/cornell\ movie-dialogs\ corpus/movie_lines.txt

echo "--conversations--"
head ./data/cornell\ movie-dialogs\ corpus/movie_conversations.txt

Couldn't find program: 'bash'


In [3]:
# data.prepare_raw_data()

In [4]:
# data.process_data()

In [5]:
def _get_buckets():
    """ Load the dataset into buckets based on their lengths.
    train_buckets_scale is the inverval that'll help us 
    choose a random bucket later on.
    """
    test_buckets = data.load_data('test_ids.enc', 'test_ids.dec')
    data_buckets = data.load_data('train_ids.enc', 'train_ids.dec')
    train_bucket_sizes = [len(data_buckets[b])
                          for b in range(len(config.BUCKETS))]
    print("Number of samples in each bucket:\n", train_bucket_sizes)
    train_total_size = sum(train_bucket_sizes)
    # list of increasing numbers from 0 to 1 that we'll use to select a bucket.
    train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                           for i in range(len(train_bucket_sizes))]
    print("Bucket scale:\n", train_buckets_scale)
    return test_buckets, data_buckets, train_buckets_scale

test_buckets, data_buckets, train_buckets_scale = _get_buckets()

FileNotFoundError: [Errno 2] No such file or directory: 'processed\\test_ids.enc'

In [ ]:
config.BUCKETS

In [ ]:
data_buckets[0]

In [ ]:
bucket_id = 0

encoder_inputs, decoder_inputs, decoder_masks = data.get_batch(
                data_buckets[bucket_id], bucket_id, batch_size=config.BATCH_SIZE)

In [ ]:
# print(encoder_inputs)
print(len(decoder_inputs))

print(config.BUCKETS[0][1])

In [ ]:
import tensorflow as tf
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import tensor_array_ops

tf.reset_default_graph()

encoder_length_tensor = tf.placeholder(tf.int32, shape=(), name='encoder_len')
encoder_inputs_tensor = tf.placeholder(tf.int32,
                                       shape=[config.BUCKETS[0][0], config.BATCH_SIZE],
                                       name='encoder_inputs')
decoder_length_tensor = tf.placeholder(tf.int32, shape=(), name='decoder_len')
decoder_inputs_tensor = tf.placeholder(tf.int32,
                                       shape=[config.BUCKETS[0][1], config.BATCH_SIZE],
                                       name='decoder_inputs')

with tf.variable_scope('encoder') as scope:
    scope = tf.get_variable_scope()
    scope.set_initializer(tf.random_uniform_initializer(-0.1,0.1))

    W = tf.get_variable(
        name="W",
        shape=[config.ENC_VOCAB, config.HIDDEN_SIZE],
        initializer=tf.random_uniform_initializer(-0.1, 0.1))

    source_embedded = tf.nn.embedding_lookup(W, encoder_inputs_tensor)

    cell = tf.nn.rnn_cell.GRUCell(num_units=config.HIDDEN_SIZE)

    enc_outputs, enc_final_state = tf.nn.dynamic_rnn(cell=cell,
                                                     inputs=source_embedded,
                                                     time_major=True,
                                                     dtype=tf.float32)

CONTEXT_SIZE = 256
with tf.variable_scope('decoder') as scope:
    scope = tf.get_variable_scope()
    scope.set_initializer(tf.random_uniform_initializer(-0.1,0.1))
    
    W = tf.get_variable(
        name="W",
        shape=[config.DEC_VOCAB, config.HIDDEN_SIZE],
        initializer=tf.random_uniform_initializer(-0.1, 0.1))
    target_embedded = tf.nn.embedding_lookup(W, decoder_inputs_tensor)
    
    cell = tf.nn.rnn_cell.GRUCell(num_units=config.HIDDEN_SIZE)
    
    def condition(time, all_outputs, inputs, states):
        return time < decoder_length_tensor
    
    def body(time, all_outputs, inputs, states):
        dec_outputs, dec_state = cell(inputs=inputs, state=states)
        
        # calculate attention
        att_keys = tf.contrib.layers.fully_connected(inputs=enc_outputs, num_outputs=CONTEXT_SIZE, activation_fn=None)
        att_query = tf.contrib.layers.fully_connected(inputs=dec_outputs, num_outputs=CONTEXT_SIZE, activation_fn=None)
        scores = tf.reduce_sum(att_keys * tf.expand_dims(att_query, 0), [2])
        scores_normalized = tf.nn.softmax(scores, dim=0)
        context = tf.expand_dims(scores_normalized, 2) * enc_outputs
        context = tf.reduce_sum(context, [0], name="context")
        print(dec_outputs.get_shape())
        print(att_keys.get_shape())
        print(att_query.get_shape())
        print(scores.get_shape())
        print(scores_normalized.get_shape())
        print(context.get_shape())
        
        projection_input = tf.concat([dec_outputs, context], 1)
        print("projection_input:", projection_input.get_shape())
        output_logits = tf.contrib.layers.fully_connected(inputs=projection_input, num_outputs=config.DEC_VOCAB, activation_fn=None)
        print("output_logits:", output_logits.get_shape())
        all_outputs = all_outputs.write(time, output_logits)
        
        next_input = tf.concat([target_embedded[time], context], 1)
        
        return (time + 1, all_outputs, next_input, dec_state)
    
    output_ta = tensor_array_ops.TensorArray(dtype=tf.float32,
                                             size=0,
                                             dynamic_size=True,
                                             element_shape=(config.BATCH_SIZE, config.DEC_VOCAB))
    
    init_inputs = tf.concat([target_embedded[0], tf.zeros(dtype=tf.float32, shape=(config.BATCH_SIZE, CONTEXT_SIZE))], 1)
    print("init_inputs:", init_inputs.get_shape())
    
    res = control_flow_ops.while_loop(
        condition,
        body,
        loop_vars=[0, output_ta, init_inputs, enc_final_state],        
    )
    final_outputs = res[1].stack()
    final_state = res[3]
    
with tf.variable_scope('loss') as scope:
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=final_outputs, labels=decoder_inputs_tensor)
    loss = tf.reduce_mean(losses)
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [ ]:
bucket_id = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for it in range(1000):
        encoder_inputs, decoder_inputs, decoder_masks = data.get_batch(
                    data_buckets[bucket_id], bucket_id, batch_size=config.BATCH_SIZE)

        res_loss, _ = sess.run([loss, train_op],
                 feed_dict={encoder_inputs_tensor: encoder_inputs,
                            decoder_inputs_tensor: decoder_inputs,
                            decoder_length_tensor: 10})
        print("Iteration {} - loss:{}".format(it, res_loss))

In [ ]:
# add perplexity calculation
# add visualization
# running model

In [ ]:
print(sum(map(np.sum, decoder_masks)))
print(sum(map(len, decoder_masks)))
print(len(data_buckets[bucket_id]))